# Project — Part I (Bootstrapping Swap Curves) #

In the IR Data.xlsm spreadsheeet, OIS data is provided. Bootstrap the
OIS discount factor Do(0, T ) and plot the discount curve for T ∈ [0, 30].

* Day Count Convention	30/360
* O/N Leg Frequency	Daily
* Fixed Leg Frequency	Annual


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# handy it seems
# https://docs.sympy.org/latest/modules/solvers/solvers.html
from sympy.solvers import solve
from sympy import Symbol


# read data
ois_data = pd.read_csv("../data/OIS_Data.csv")
irs_data = pd.read_csv("../data/IRS_Data.csv")
ois_data.columns = map(str.lower, ois_data.columns)
irs_data.columns = map(str.lower, irs_data.columns)

# use dict comprehension?
tenor_mapping = {
    "6m": 0.5,
    "1y": 1.0,
    "2y": 2.0,
    "3y": 3.0,
    "4y": 4.0,
    "5y": 5.0,
    "7y": 7.0,
    "10y": 10.0,
    "15y": 15.0,
    "20y": 20.0,
    "30y": 30.0,
}

# OIS processing
ois_data["tenor"] = ois_data["tenor"].map(tenor_mapping)
ois_data["rate"] = ois_data["rate"].str.strip("%").astype(float) / 100.0

# DAY COUNT CONVENTION IS 30/360
FULL_YEAR = 360

In [2]:
ois_data["tenor_diff"] = ois_data["tenor"] - ois_data["tenor"].shift()

##  solve for < 1 y stuff ##

Because these 2 are anchors i believe

In [3]:
f_6m = Symbol("f_6m")
f_6m = solve((1 + 0.5 * ois_data.loc[0, "rate"]) ** (1 / 180) - (1 + f_6m / 360))[0]
print(f_6m)

0.00249844747059136


In [4]:
f_1y = Symbol("f_1y")
f_1y = solve(
    ((1 + 1 * ois_data.loc[1, "rate"]) / (1 + f_6m / 360) ** 180) ** (1 / 180)
    - (1 + f_1y / 360)
)[0]
print(f_1y)

0.00349259610064934


In [5]:
ois_data.loc[0, "disc_factor"] = 1 / (1 + f_6m / 360) ** 180
ois_data.loc[1, "disc_factor"] = 1 / (
    ((1 + f_6m / 360) ** 180) * ((1 + f_1y / 360) ** 180)
)

In [6]:
ois_data

,tenor,product,rate,tenor_diff,disc_factor
0,0.5,OIS,0.00250,NaN,0.998751560549318
1,1.0,OIS,0.00300,0.5,0.997008973080741
2,2.0,OIS,0.00325,1.0,NaN
3,3.0,OIS,0.00335,1.0,NaN
4,4.0,OIS,0.00350,1.0,NaN
5,5.0,OIS,0.00360,1.0,NaN
6,7.0,OIS,0.00400,2.0,NaN
7,10.0,OIS,0.00450,3.0,NaN
8,15.0,OIS,0.00500,5.0,NaN
9,20.0,OIS,0.00525,5.0,NaN


## Let's try the collapsing shit, thx eko ##
I can test up until 5 first

In [7]:
y_1_funny = (
    ((1 + f_6m / 360) ** 180)
    * ((1 + f_1y / 360) ** 180)
    * ois_data.loc[1, "disc_factor"]
)
# youd be surprised
print(y_1_funny)

1.00000000000000


In [8]:
for k in range(2, 6):
    denominator = 1 / (1 + 1 * ois_data.loc[k, "rate"])
    sum_prev_disc_rate = ois_data.loc[1:k, "disc_factor"].sum()
    # print(sum_prev_disc_rate)
    numerator = y_1_funny - 1 * ois_data.loc[k, "rate"] * sum_prev_disc_rate
    ois_data.loc[k, "disc_factor"] = numerator * denominator

## interpolation required ##

In [9]:
previous_disc_factors = ois_data.loc[1:5, "disc_factor"].to_numpy()
print(previous_disc_factors)

[0.997008973080741 0.993530745913270 0.990015141218289 0.986116649715253
 0.982184119733221]


In [10]:
# k = 6 to 10; try alternative method
for k in range(6, 11):
    mul_fact = 1.0 / ois_data.loc[k, "tenor_diff"]
    rate = ois_data.loc[k, "rate"]
    max_sum = sum(range(1, int(ois_data.loc[k, "tenor_diff"])))
    # print(f"{mul_fact} * {max_sum} = {mul_fact * max_sum}")
    regressor = mul_fact * max_sum
    pv_fix_wo_unknown = (
        previous_disc_factors.sum() + regressor * previous_disc_factors[-1]
    )
    pv_float_wo_unknown = y_1_funny
    x = Symbol("x")
    tosolve = solve(
        rate * 1 * (pv_fix_wo_unknown + regressor * x + x) - (pv_float_wo_unknown - x)
    )
    # print(tosolve[0])
    ois_data.loc[k, "disc_factor"] = tosolve[0]

    # find points between
    interpolated_disc_factors = []
    diff_factor = mul_fact * (
        ois_data.loc[k, "disc_factor"] - ois_data.loc[k - 1, "disc_factor"]
    )

    for x in range(1, int(ois_data.loc[k, "tenor_diff"])):
        to_append = ois_data.loc[k - 1, "disc_factor"] + diff_factor * x
        # print(f"{x} = {to_append}")
        interpolated_disc_factors.append(to_append)
    # append them
    previous_disc_factors = np.append(previous_disc_factors, interpolated_disc_factors)
    previous_disc_factors = np.append(
        previous_disc_factors, ois_data.loc[k, "disc_factor"]
    )

In [11]:
ois_disc_factors = np.append(ois_data["disc_factor"].values[0], previous_disc_factors)
ois_tenors = np.append([0.5], np.arange(1, 31, 1))
df_ois_df = pd.DataFrame({"tenor": ois_tenors, "ois_df": ois_disc_factors})

In [12]:
df_ois_df

,tenor,ois_df
0,0.5,0.998751560549318
1,1.0,0.997008973080741
2,2.0,0.993530745913270
3,3.0,0.990015141218289
4,4.0,0.986116649715253
5,5.0,0.982184119733221
6,6.0,0.977294947163773
7,7.0,0.972405774594324
8,8.0,0.966929475904951
9,9.0,0.961453177215578


### finally OIS data

In [13]:
ois_data

,tenor,product,rate,tenor_diff,disc_factor
0,0.5,OIS,0.00250,NaN,0.998751560549318
1,1.0,OIS,0.00300,0.5,0.997008973080741
2,2.0,OIS,0.00325,1.0,0.993530745913270
3,3.0,OIS,0.00335,1.0,0.990015141218289
4,4.0,OIS,0.00350,1.0,0.986116649715253
5,5.0,OIS,0.00360,1.0,0.982184119733221
6,7.0,OIS,0.00400,2.0,0.972405774594324
7,10.0,OIS,0.00450,3.0,0.955976878526205
8,15.0,OIS,0.00500,5.0,0.927611479605330
9,20.0,OIS,0.00525,5.0,0.900075937041340


### 1b

In [14]:
# combined data
combined_tenors = np.arange(0.5, 30.5, 0.5)
df_combined = pd.DataFrame({"tenor": combined_tenors})
df_combined = pd.merge(df_combined, df_ois_df, how="left", on="tenor")
df_combined["ois_df"] = df_combined["ois_df"].astype(float)
df_combined.interpolate(inplace=True)
# df_combined = pd.merge(df_combined, ois_data[["tenor", "rate"]], how="left", on="tenor")
# df_combined.rename(columns={"rate": "ois_rate"}, inplace=True)

In [15]:
# read data
irs_data = pd.read_csv("../data/IRS_Data.csv")
irs_data.columns = map(str.lower, irs_data.columns)
irs_data["tenor"] = irs_data["tenor"].map(tenor_mapping)
irs_data["rate"] = irs_data["rate"].str.strip("%").astype(float) / 100.0
irs_data["tenor_diff"] = irs_data["tenor"] - irs_data["tenor"].shift()
irs_data["tenor_diff"].fillna(0, inplace=True)
irs_data["interp_count"] = (irs_data["tenor_diff"] / 0.5).astype(int)
irs_data.rename(columns={"rate": "irs_rate"}, inplace=True)

In [16]:
irs_data.loc[0, "irs_df"] = 1 / (
    1 + irs_data.loc[0, "tenor"] * irs_data.loc[0, "irs_rate"]
)
irs_data.loc[0, "fw_libor"] = irs_data.loc[0, "irs_rate"]

In [17]:
irs_data

,tenor,product,irs_rate,tenor_diff,interp_count,irs_df,fw_libor
0,0.5,LIBOR,0.0250,0.0,0,0.987654,0.025
1,1.0,IRS,0.0280,0.5,1,NaN,NaN
2,2.0,IRS,0.0300,1.0,2,NaN,NaN
3,3.0,IRS,0.0315,1.0,2,NaN,NaN
4,4.0,IRS,0.0325,1.0,2,NaN,NaN
5,5.0,IRS,0.0330,1.0,2,NaN,NaN
6,7.0,IRS,0.0350,2.0,4,NaN,NaN
7,10.0,IRS,0.0370,3.0,6,NaN,NaN
8,15.0,IRS,0.0400,5.0,10,NaN,NaN
9,20.0,IRS,0.0450,5.0,10,NaN,NaN


In [18]:
# finally something we can work with
df_combined = pd.merge(
    df_combined,
    irs_data[["tenor", "irs_rate", "fw_libor", "irs_df"]],
    how="left",
    on="tenor",
)

In [19]:
irs_df_1 = Symbol("irs_df_1")
just_try_irs_df_1 = solve(
    (
        0.5
        * (df_combined.loc[0, "ois_df"] + df_combined.loc[1, "ois_df"])
        * df_combined.loc[1, "irs_rate"]
    )
    - 0.5
    * (
        df_combined.loc[0, "ois_df"] * df_combined.loc[0, "fw_libor"]
        + df_combined.loc[1, "ois_df"]
        * 2
        * (((df_combined.loc[0, "irs_df"] - irs_df_1) / irs_df_1))
    )
)

In [20]:
# define a root search function(bisect)
def bisect_func(f, a, b, tol):
    assert a < b and f(a) * f(b) < 0
    c = (a + b) / 2
    while (b - a) / 2 > tol:
        c = (a + b) / 2
        if abs(f(c)) < tol:
            return c
        else:
            if f(a) * f(c) < 0:
                b = c
            else:
                a = c
    return c

In [21]:
def get_pv_float(x):
    start_irs_df = df_combined.loc[prev_idx_with_k, "irs_df"]
    df_list = [start_irs_df]
    # print(f"INSIDE GET FLOAT > {prev_idx_with_k} & {idx_with_k} and {pv_float_known}")
    max_count = idx_with_k - prev_idx_with_k
    if abs(max_count) > 1:
        diff = x - start_irs_df
        libor_payoff_list = []
        for n in range(1, max_count + 1):
            idx_to_use = prev_idx_with_k + n
            # print(
            #    f"INSIDE GET FLOAT > {prev_idx_with_k}- {df_list} & {idx_with_k} and {pv_float_known} and {idx_to_use}"
            # )

            mul_fact = n / (max_count)
            df_interp = start_irs_df + mul_fact * diff
            libor_payoff = (
                0.5
                * (2)
                * ((df_list[-1] - df_interp) / df_interp)
                * df_combined.loc[idx_to_use, "ois_df"]
            )
            df_list.append(df_interp)
            libor_payoff_list.append(libor_payoff)
        pv_float_unknown = sum(libor_payoff_list)

    else:
        df_interp = x
        libor_payoff = (
            0.5
            * (2)
            * ((df_list[-1] - df_interp) / df_interp)
            * df_combined.loc[1, "ois_df"]
        )
        pv_float_unknown = libor_payoff

    return pv_float_unknown + pv_float_known - pv_fix

In [22]:
# start from 1 because we already know for row 0, i.e. 6m
idx_with_k = 1
prev_idx_with_k = 0
maxrow = len(df_combined)
while idx_with_k < maxrow:
    if np.isnan(df_combined.loc[idx_with_k, "irs_rate"]):
        # print("next")
        idx_with_k = idx_with_k + 1
    else:
        # present value of fix
        pv_fix = (
            0.5
            * df_combined.loc[idx_with_k, "irs_rate"]
            * df_combined.loc[0:idx_with_k, "ois_df"].sum()
        )
        # print(df_combined.loc[0:idx_with_k, "ois_df"])

        # we try to sum up floating leg as much as possible
        pv_float_known = 0
        idx_with_fwlibor = 0
        k = 0
        while not (np.isnan(df_combined.loc[k, "fw_libor"])):
            known_float = (
                0.5 * df_combined.loc[k, "ois_df"] * df_combined.loc[k, "fw_libor"]
            )
            pv_float_known = pv_float_known + known_float
            idx_with_fwlibor = idx_with_fwlibor + 1
            k = k + 1
        # print(f"known until {k-1}")
        # refer senior for inspiration how to do a very challenging interpolation
        # print(f"{prev_idx_with_k} & {idx_with_k} and and {pv_fix} vs {pv_float_known}")
        # f = lambda x: (get_pv_float(x))
        f = lambda x: (get_pv_float(x))
        df_last = bisect_func(f, 1e-10, 1, 1e-10)
        # print(df_last)

        # fill in value !
        max_count = idx_with_k - prev_idx_with_k
        if (max_count) > 1:
            for n in range(prev_idx_with_k + 1, idx_with_k + 1):
                start_irs_df = df_combined.loc[prev_idx_with_k, "irs_df"]
                diff = df_last - start_irs_df
                # print(f"start from {start_irs_df} with diff {diff}")
                mul_fact = (n - prev_idx_with_k) / (max_count)
                df_interp = start_irs_df + mul_fact * diff
                # print(n)
                df_combined.loc[n, "irs_df"] = df_interp
                df_combined.loc[n, "fw_libor"] = 2 * (
                    df_combined.loc[n - 1, "irs_df"] / df_combined.loc[n, "irs_df"] - 1
                )
                # print("filled !")

        else:
            df_combined.loc[1, "irs_df"] = df_last
            df_combined.loc[1, "fw_libor"] = 2 * (
                df_combined.loc[0, "irs_df"] / df_combined.loc[1, "irs_df"] - 1
            )

        prev_idx_with_k = idx_with_k
        idx_with_k = idx_with_k + 1

In [23]:
irs_data.drop(columns=["irs_df", "fw_libor"], inplace=True)
irs_data = pd.merge(irs_data, df_combined, how="left", on="tenor")
irs_data.drop(columns=["irs_rate_y"], inplace=True)

### For 1c is NOT FRA, this is "find par rate for forward starting collateralized IRS"

In [24]:
df_1c = pd.DataFrame()

In [25]:
maturity = np.append(np.append(np.ones(5), 5 * np.ones(5)), 10 * np.ones(5))
duration = [1, 2, 3, 5, 10] * 3

In [26]:
df_1c["maturity"] = maturity
df_1c["duration"] = duration
df_1c["maturity"] = df_1c["maturity"].astype(float)
df_1c["duration"] = df_1c["duration"].astype(float)

In [27]:
for x, row in df_1c.iterrows():
    maturity = df_1c.loc[x, "maturity"]
    duration = df_1c.loc[x, "duration"]
    time_sum = maturity + duration
    rows_to_work = df_combined[
        (df_combined["tenor"] >= maturity) & (df_combined["tenor"] <= time_sum)
    ]
    # drop first row because the first payment takes place like 6m after the maturity ends
    # e.g. if maturity = 1y, then first payment = 1.5y
    rows_to_work = rows_to_work.drop(rows_to_work.index[[0]])
    df_1c.loc[x, "pv_fix_nok"] = 0.5 * rows_to_work["ois_df"].sum()
    df_1c.loc[x, "pv_float"] = (
        0.5 * (rows_to_work["ois_df"] * rows_to_work["fw_libor"]).sum()
    )
    df_1c.loc[x, "k_rate"] = df_1c.loc[x, "pv_float"] / df_1c.loc[x, "pv_fix_nok"]
    print(rows_to_work)

   tenor    ois_df  irs_rate  fw_libor    irs_df
2    1.5  0.995270       NaN  0.031751  0.957378
3    2.0  0.993531      0.03  0.032263  0.942179
   tenor    ois_df  irs_rate  fw_libor    irs_df
2    1.5  0.995270       NaN  0.031751  0.957378
3    2.0  0.993531    0.0300  0.032263  0.942179
4    2.5  0.991773       NaN  0.034218  0.926330
5    3.0  0.990015    0.0315  0.034814  0.910482
   tenor    ois_df  irs_rate  fw_libor    irs_df
2    1.5  0.995270       NaN  0.031751  0.957378
3    2.0  0.993531    0.0300  0.032263  0.942179
4    2.5  0.991773       NaN  0.034218  0.926330
5    3.0  0.990015    0.0315  0.034814  0.910482
6    3.5  0.988066       NaN  0.035207  0.894731
7    4.0  0.986117    0.0325  0.035838  0.878981
    tenor    ois_df  irs_rate  fw_libor    irs_df
2     1.5  0.995270       NaN  0.031751  0.957378
3     2.0  0.993531    0.0300  0.032263  0.942179
4     2.5  0.991773       NaN  0.034218  0.926330
5     3.0  0.990015    0.0315  0.034814  0.910482
6     3.5  0.98

# FINALLY, SUMMARY ANSWERS FOR PART 1

In [28]:
# 1A. OIS CURVEBUILDING
ois_data

,tenor,product,rate,tenor_diff,disc_factor
0,0.5,OIS,0.00250,NaN,0.998751560549318
1,1.0,OIS,0.00300,0.5,0.997008973080741
2,2.0,OIS,0.00325,1.0,0.993530745913270
3,3.0,OIS,0.00335,1.0,0.990015141218289
4,4.0,OIS,0.00350,1.0,0.986116649715253
5,5.0,OIS,0.00360,1.0,0.982184119733221
6,7.0,OIS,0.00400,2.0,0.972405774594324
7,10.0,OIS,0.00450,3.0,0.955976878526205
8,15.0,OIS,0.00500,5.0,0.927611479605330
9,20.0,OIS,0.00525,5.0,0.900075937041340


In [29]:
# 1B. IRS CURVEBUILDING
irs_data

,tenor,product,irs_rate_x,tenor_diff,interp_count,ois_df,fw_libor,irs_df
0,0.5,LIBOR,0.0250,0.0,0,0.998752,0.025000,0.987654
1,1.0,IRS,0.0280,0.5,1,0.997009,0.031005,0.972577
2,2.0,IRS,0.0300,1.0,2,0.993531,0.032263,0.942179
3,3.0,IRS,0.0315,1.0,2,0.990015,0.034814,0.910482
4,4.0,IRS,0.0325,1.0,2,0.986117,0.035838,0.878981
5,5.0,IRS,0.0330,1.0,2,0.982184,0.035326,0.848989
6,7.0,IRS,0.0350,2.0,4,0.972406,0.041298,0.784216
7,10.0,IRS,0.0370,3.0,6,0.955977,0.044033,0.692710
8,15.0,IRS,0.0400,5.0,10,0.927611,0.051401,0.551081
9,20.0,IRS,0.0450,5.0,10,0.900076,0.069994,0.408218


In [30]:
# 1C. FORWARD STARTING INTEREST RATE SWAPS
df_1c

,maturity,duration,pv_fix_nok,pv_float,k_rate
0,1.0,1.0,0.994400,0.031828,0.032007
1,1.0,2.0,1.985294,0.066029,0.033259
2,1.0,3.0,2.972386,0.101093,0.034011
3,1.0,5.0,4.934070,0.173953,0.035255
4,1.0,10.0,9.747887,0.374591,0.038428
5,5.0,1.0,0.978517,0.038430,0.039274
6,5.0,2.0,1.952145,0.078232,0.040075
7,5.0,3.0,2.920444,0.117029,0.040072
8,5.0,5.0,4.840612,0.198916,0.041093
9,5.0,10.0,9.542492,0.416374,0.043634


In [31]:
## export to csv
df_1c.to_csv("df_1c.csv", index=False)